In [7]:
# Load the data and libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

frisbee = pd.read_csv('https://raw.githubusercontent.com/jbellizia/cs-3110-final/refs/heads/main/ultimate_college_championship.csv')

In [8]:
frisbee

,player,level,gender,division,team_name,Turns,Ds,Assists,Points,plus_minus,team_games,turns_per_game,ds_per_game,ast_per_game,pts_per_game,pls_mns_per_game
0,Jacques Nissen,Division 1,Men,Division 1 Men,Brown Brownian Motion,12,8,38,13,47,8,1.500000,1.000000,4.750000,1.625000,5.875000
1,Cal Nightingale,Division 1,Men,Division 1 Men,Brown Brownian Motion,3,0,12,27,36,8,0.375000,0.000000,1.500000,3.375000,4.500000
2,Faye Burdick,Division 1,Women,Division 1 Women,Colorado Quandary,6,12,16,13,35,7,0.857143,1.714286,2.285714,1.857143,5.000000
3,Matthew Gregor,Division 3,Men,Division 3 Men,Franciscan Fatal,2,6,3,26,33,6,0.333333,1.000000,0.500000,4.333333,5.500000
4,Frankie Saraniti,Division 3,Women,Division 3 Women,Carleton Eclipse,11,15,12,17,33,6,1.833333,2.500000,2.000000,2.833333,5.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1660,Lily van Linder,Division 3,Women,Division 3 Women,Occidental WAC,69,19,8,9,-33,6,11.500000,3.166667,1.333333,1.500000,-5.500000
1661,Lanie O'Neill,Division 3,Women,Division 3 Women,Davenport Panthers,74,11,21,3,-39,5,14.800000,2.200000,4.200000,0.600000,-7.800000
1662,Bella Steedly,Division 3,Women,Division 3 Women,Wellesley Whiptails,82,21,15,5,-41,6,13.666667,3.500000,2.500000,0.833333,-6.833333
1663,Rui Lin,Division 3,Women,Division 3 Women,Grinnell Grinneleanor Roosevelts,55,7,3,0,-45,6,9.166667,1.166667,0.500000,0.000000,-7.500000
